## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-09-11-43-38 +0000,nyt,"There’s Barely Any Food in Gaza, and Barely An...",https://www.nytimes.com/2025/08/09/world/middl...
1,2025-08-09-11-14-40 +0000,wapo,"Trump, Putin meeting set for next week in Alaska",https://www.washingtonpost.com/politics/2025/0...
2,2025-08-09-11-00-00 +0000,latimes,L.A. County fire captain faked work injury to ...,https://www.latimes.com/california/story/2025-...
3,2025-08-09-10-49-57 +0000,nyt,Russia Goes After Ukraine With Distant Strikes...,https://www.nytimes.com/2025/08/09/world/europ...
4,2025-08-09-10-25-42 +0000,nyt,Zelensky Rejects Trump’s Suggestion That Ukrai...,https://www.nytimes.com/2025/08/09/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2357/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
7,trump,77
4,gaza,22
200,plan,16
26,new,15
70,city,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
97,2025-08-09-01-00-00 +0000,wsj,President Trump is considering reclassifying m...,https://www.wsj.com/politics/policy/trump-mari...,194
243,2025-08-08-16-13-00 +0000,wsj,President Trump has directed the Pentagon to b...,https://www.wsj.com/politics/national-security...,182
50,2025-08-09-07-51-56 +0000,bbc,Mexico rejects Trump's reported military plan ...,https://www.bbc.com/news/articles/c07pdel9vyjo...,139
200,2025-08-08-19-22-00 +0000,wsj,President Trump’s team is reviewing new conten...,https://www.wsj.com/economy/central-banking/tr...,139
245,2025-08-08-16-06-14 +0000,nypost,Trump secretly authorized military force agai...,https://nypost.com/2025/08/08/us-news/trump-se...,133


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
97,194,2025-08-09-01-00-00 +0000,wsj,President Trump is considering reclassifying m...,https://www.wsj.com/politics/policy/trump-mari...
243,89,2025-08-08-16-13-00 +0000,wsj,President Trump has directed the Pentagon to b...,https://www.wsj.com/politics/national-security...
192,82,2025-08-08-20-03-13 +0000,nypost,Inside Israel’s plan to take over Gaza City an...,https://nypost.com/2025/08/08/world-news/insid...
18,56,2025-08-09-09-48-43 +0000,nypost,Zelensky vows Russia will not get any Ukrainia...,https://nypost.com/2025/08/09/us-news/zelensky...
258,54,2025-08-08-14-53-06 +0000,nypost,Longtime foes Armenia and Azerbaijan ink histo...,https://nypost.com/2025/08/08/us-news/armenia-...
161,43,2025-08-08-21-26-21 +0000,nypost,Officer killed in shooting outside CDC HQ in A...,https://nypost.com/2025/08/08/us-news/active-s...
160,43,2025-08-08-21-29-49 +0000,nypost,Trump admin considering two new candidates to ...,https://nypost.com/2025/08/08/business/trump-a...
234,40,2025-08-08-16-47-10 +0000,nypost,JD Vance goes fishing with top UK official Dav...,https://nypost.com/2025/08/08/us-news/jd-vance...
155,38,2025-08-08-21-32-18 +0000,nypost,Embattled Intel CEO calls out ‘misinformation’...,https://nypost.com/2025/08/08/business/intel-c...
158,33,2025-08-08-21-31-22 +0000,nypost,Vicious NYC killing of 14-year-old Angel Mendo...,https://nypost.com/2025/08/08/us-news/vicious-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
